In [1]:
from sshtunnel import SSHTunnelForwarder
from psycopg2.extras import execute_values
import psycopg2, os, logging
import pandas as pd
from utils import *

In [2]:
# Navigate to the secret file to gain access to the AWS EC2 instance
parent_directory = "C:\\Users\\kazir\\Desktop\\Projects\\nerdy_pick"
db_secret_file = "db\\private-subnet-db.pem"
secret_path = os.path.join(parent_directory, db_secret_file)
secret_path

'C:\\Users\\kazir\\Desktop\\Projects\\nerdy_pick\\db\\private-subnet-db.pem'

In [3]:
# Create a logging file to keep track of bugs and execution
log_path = os.path.join(parent_directory, "logs\\sportsbookdb.log")
logging.basicConfig(
    filename=log_path,
    filemode="a",
    level=logging.INFO,
    format= "%(asctime)s [%(levelname)s] - %(message)s",
    datefmt= "%d-%b-%y %H:%M:%S",
)

In [4]:
# SSH credentials
SSH_HOST = os.environ["SSH_ENDPOINT"]
SSH_PORT = 22
SSH_USER = os.environ["SSH_USER"]
SSH_PRIVATE_KEY = secret_path
logging.log(level=logging.INFO, msg="SSH credentials imported")

# Database credentials
DB_HOST = os.environ["DB_ENDPOINT"]
DB_PORT = 5432
DB_USERNAME = os.environ["DB_USERNAME"]
DB_PASSWORD = os.environ["DB_PASSWORD"]
DB_NAME = os.environ["DB_NAME"]
logging.log(level=logging.INFO, msg="PostgreSQL credentials imported")

In [5]:
def connect_to_ssh_db(db_name: str = DB_NAME) -> tuple[psycopg2.connect, SSHTunnelForwarder]:
    """
    Connects to the Jump Box on AWS using SSH tunneling and also connects with the database

    Params:
        db_name (str): Name of the database to connect to
    
    Returns:
        tunnel, conn: SSH tunnel and connection to the database
    """
    # Set up SSH tunnel
    tunnel = SSHTunnelForwarder(
        (SSH_HOST, SSH_PORT),
        ssh_username=SSH_USER,
        ssh_pkey=SSH_PRIVATE_KEY,
        remote_bind_address=(DB_HOST, DB_PORT)
    )
    logging.log(level=logging.INFO, msg="Sucessfully connected to SSH tunnel")
    # Start the tunnel
    tunnel.start()

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host='127.0.0.1',
        port=tunnel.local_bind_port,
        user=DB_USERNAME,
        password=DB_PASSWORD,
        dbname=DB_NAME
    )
    logging.log(level=logging.INFO, msg="Connected to PostgreSQL")
    return (tunnel, conn)

In [6]:
# Create a connection
tunnel, conn = connect_to_ssh_db()

In [7]:
# Query the database
connection = conn.cursor()
connection.execute("SELECT * FROM prizepicks_prop")
results = connection.fetchall()
pd.DataFrame(results)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3318980,218936,2024-11-13,19:40:00,Points,Standard,Not Combo,Brandon Boston,NBA,G,NOP,OKC,15.5,Not Adjusted,200,2024-11-12
1,3319028,218936,2024-11-13,19:40:00,Points + Rebounds + Assists,Standard,Not Combo,Brandon Boston,NBA,G,NOP,OKC,24.5,Not Adjusted,200,2024-11-12
2,3318981,79951,2024-11-13,19:40:00,Points,Standard,Not Combo,Jalen Williams,NBA,G-F,OKC,NOP,20.5,Not Adjusted,201,2024-11-12
3,3319006,79951,2024-11-13,19:40:00,Rebounds,Standard,Not Combo,Jalen Williams,NBA,G-F,OKC,NOP,6.5,Not Adjusted,201,2024-11-12
4,3319026,79951,2024-11-13,19:40:00,Three Pointers Made,Standard,Not Combo,Jalen Williams,NBA,G-F,OKC,NOP,1.5,Not Adjusted,201,2024-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,3325416,211037,2024-11-17,16:05:00,Longest Rush,Standard,Not Combo,Geno Smith,NFL,QB,SEA,SF,8.5,Not Adjusted,18,2024-11-03
2899,3325417,214664,2024-11-17,16:05:00,Longest Rush,Standard,Not Combo,Kenneth Walker III,NFL,RB,SEA,SF,13.5,Not Adjusted,19,2024-11-03
2900,3325399,211234,2024-11-17,20:20:00,Longest Rush,Standard,Not Combo,Chase Brown,NFL,RB,CIN,LAC,14.5,Not Adjusted,20,2024-11-03
2901,3325400,211419,2024-11-17,20:20:00,Longest Rush,Standard,Not Combo,J.K. Dobbins,NFL,RB,LAC,CIN,14.5,Not Adjusted,21,2024-11-03


In [8]:
# Import the player and prop datas to import into the database
players_data_nfl, lines_data_nfl = prizepicks_lines("NFL")
players_data_nba, lines_data_nba = prizepicks_lines("NBA")
all_players_data = players_data_nba + players_data_nfl
all_props_data = lines_data_nba + lines_data_nfl
logging.log(level=logging.INFO, msg=f"{len(all_players_data)} players and {len(all_props_data)} prop datas found on Prizepicks")

In [9]:
# Queries to import data into the database
prizepicks_player_query = """
    INSERT INTO prizepicks_player (player_id, player_name, player_pos, player_team_abbr, player_team, player_combo, player_league)
    VALUES %s
    ON CONFLICT (player_id) DO NOTHING;
"""

prizepicks_prop_query = """
    INSERT INTO prizepicks_prop (prop_id, player_id, game_date, game_time, prop_desc, prop_type, prop_combo, prop_name, prop_league, player_pos, player_team, opponent_team, prop_line, prop_adjusted, prop_rank, board_date)
    VALUES %s
    ON CONFLICT (prop_id) DO NOTHING;
"""

In [10]:
def import_data_into_db(connection: psycopg2.connect, query: str, data: list) -> str:
    """
    Handles appending new data into the PostgreSQL database and commiting the changes

    Params:
        connection (psycopg2.connect): Database connection
        query (str): SQL query to insert data into a database
        data (list): A list of tuple values
    """
    query_type = "players query" if query == prizepicks_player_query else "props query"
    cursor = connection.cursor()
    execute_values(cur=cursor, sql=query, argslist=data)
    logging.log(level=logging.INFO, msg=f"Successfully ran the SQL {query_type} to import data into the database")
    connection.commit()
    logging.log(level=logging.INFO, msg=f"Commited the changes into the PostgreSQL database")
    return print("Success!")

In [11]:
# Player and prop datas to append into the database
import_data_into_db(connection=conn, query=prizepicks_player_query, data=all_players_data)
import_data_into_db(connection=conn, query=prizepicks_prop_query, data=all_props_data)

Success!
Success!


In [12]:
# Close both connections
tunnel.close()
logging.log(level=logging.INFO, msg="Closed and disconnected the SSH tunnel")
conn.close()
logging.log(level=logging.INFO, msg="Closed and disconnected the database connection")